In [1]:
import timeit
import csv
import MinCompSpin as mod

In [2]:
def write_results(save_path, results):
    with open(save_path, "w", newline='') as csv_file:
        csv_writer = csv.writer(csv_file)
        for row in results:
            csv_writer.writerow(row)

debug = True

In [3]:
# The base data sets
data_path_prefix = "MinCompSpin_Greedy/INPUT/"
output_path_prefix = "MinCompSpin_Greedy/OUTPUT/"

Ns = [1000, 750, 500, 250, 100, 10]
ns = [100, 60, 40, 20]
data_sets = []
def add_data_set(n, N):
    name = f"my_data_n{n}_N{N}.dat"
    path = data_path_prefix + name
    Kset = mod.read_datafile(path, n)
    assert(Kset.N == N)
    Partitions = mod.GreedySearch(Kset)
    data_sets.append([Kset, Partitions, n, N, name])
add_data_set(100, 2000)
for N in Ns:
    for n in ns:
        add_data_set(n, N)

print(f"\nTotal of {len(data_sets)} data sets\n")
print(data_sets)

Read the dataset: MinCompSpin_Greedy/INPUT/my_data_n100_N2000.dat
Number of variables to read: n = 100
	 **** Filling in the rxr-triangular Matrix ****
	 **** Start Merging ****
	 **** Greedy Merging Finished ****

Read the dataset: MinCompSpin_Greedy/INPUT/my_data_n100_N1000.dat
Number of variables to read: n = 100
	 **** Filling in the rxr-triangular Matrix ****
	 **** Start Merging ****
	 **** Greedy Merging Finished ****

Read the dataset: MinCompSpin_Greedy/INPUT/my_data_n60_N1000.dat
Number of variables to read: n = 60
	 **** Filling in the rxr-triangular Matrix ****
	 **** Start Merging ****
	 **** Greedy Merging Finished ****

Read the dataset: MinCompSpin_Greedy/INPUT/my_data_n40_N1000.dat
Number of variables to read: n = 40
	 **** Filling in the rxr-triangular Matrix ****
	 **** Start Merging ****
	 **** Greedy Merging Finished ****

Read the dataset: MinCompSpin_Greedy/INPUT/my_data_n20_N1000.dat
Number of variables to read: n = 20
	 **** Filling in the rxr-triangular Matrix

# Test 1
This tests the greedy search on the SCOTUS n9 data set.

In [4]:
test1_results = []
def test1():
    data_path = "MinCompSpin_Greedy/INPUT/SCOTUS_n9_N895_Data.dat"
    setup = f'''
import MinCompSpin as mod
def test():
    Kset = mod.read_datafile("{data_path}", 9)

    fp1 = mod.GreedySearch(Kset)
    S = mod.Entropy(Kset)
    LogL_g = mod.LogL_MCM(Kset, fp1)
    LogE_g = mod.LogE_MCM(Kset, fp1)
    LogL_infoICC = mod.LogL_MCM_infoICC(Kset, fp1)
    LogE_infoICC = mod.LogE_MCM_infoICC(Kset, fp1)
'''
    stmt = 'test()'

    repeat = 5
    number = 1000
    if debug:
         number = 1
    test1_results = timeit.repeat(stmt, setup, repeat=repeat, number=number)
    write_results(data_path + ".test1.dat", [test1_results])
test1()

Read the dataset: MinCompSpin_Greedy/INPUT/SCOTUS_n9_N895_Data.dat
Number of variables to read: n = 9
	 **** Filling in the rxr-triangular Matrix ****
	 **** Start Merging ****
	 **** Greedy Merging Finished ****

Complete model: 111111111	  LogE(CM) = -3305.547	 = -5.328 bits per datapoint

ICC 1: 	 010001011	 LogE(ICC) = -1545.984	 = -2.492 bits per datapoint 	
ICC 2: 	 101110100	 LogE(ICC) = -1754.412	 = -2.828 bits per datapoint 	

Log-Evidence(MCM) = -3300.395	 = -5.320 bits per datapoint 	
Complete model: 111111111	  LogE(CM) = -3305.547	 = -5.328 bits per datapoint

ICC 1: 	 010001011	 LogE(ICC) = -1545.984	 = -2.492 bits per datapoint 	
ICC 2: 	 101110100	 LogE(ICC) = -1754.412	 = -2.828 bits per datapoint 	

Log-Evidence(MCM) = -3300.395	 = -5.320 bits per datapoint 	
Read the dataset: MinCompSpin_Greedy/INPUT/SCOTUS_n9_N895_Data.dat
Number of variables to read: n = 9
	 **** Filling in the rxr-triangular Matrix ****
	 **** Start Merging ****
	 **** Greedy Merging Finished ****

# Test 2
This tests the overhead of python calls and the conversion of data types. It does so for every data set used.

In [5]:
test2_results = []
def test2(data_set_idx):
    data_set = data_sets[data_set_idx]
    data_path = data_path_prefix + data_set[4]
    n = data_set[2]
    setup = f'''
import MinCompSpin as mod
Kset = data_sets[{data_set_idx}][0]
Partitions = data_sets[{data_set_idx}][1]
def test_Kset():
    mod.test_convert_Kset(Kset)
    
def test_Partitions():
    mod.test_convert_Partitions(Partitions)
'''

    repeat = 10
    number = 10
    if debug:
         number = 2
    test2_results = []
    for name, stmt in [('Kset', 'test_Kset()'), ('Partitions', 'test_Partitions')]:
        test2_results.append([name] + list(timeit.repeat(stmt, setup, repeat=repeat, number=number, globals=globals())))
    write_results(output_path_prefix + data_set[4] + ".test2", test2_results)

for data_set_idx in range(len(data_sets)):
    test2(data_set_idx)

# Test 3
This tests the Greedy Search on data sets of different sizes.

In [6]:
test3_results = []

def test3(data_set_idx):
    data_set = data_sets[data_set_idx]
    data_path = data_path_prefix + data_set[4]
    n = data_set[2]
    setup = f'''
import MinCompSpin as mod
Kset = data_sets[{data_set_idx}][0]
'''
    stmt = 'mod.GreedySearch(Kset)'

    repeat = 10
    number = 10
    if debug:
         number = 2
    test3_results = timeit.repeat(stmt, setup, repeat=repeat, number=number, globals=globals())
    write_results(output_path_prefix + data_set[4] + ".test3", [test3_results])

for data_set_idx in range(len(data_sets)):
    test3(data_set_idx)

	 **** Filling in the rxr-triangular Matrix ****
	 **** Start Merging ****
	 **** Greedy Merging Finished ****

	 **** Filling in the rxr-triangular Matrix ****
	 **** Start Merging ****
	 **** Greedy Merging Finished ****

	 **** Filling in the rxr-triangular Matrix ****
	 **** Start Merging ****
	 **** Greedy Merging Finished ****

	 **** Filling in the rxr-triangular Matrix ****
	 **** Start Merging ****
	 **** Greedy Merging Finished ****

	 **** Filling in the rxr-triangular Matrix ****
	 **** Start Merging ****
	 **** Greedy Merging Finished ****

	 **** Filling in the rxr-triangular Matrix ****
	 **** Start Merging ****
	 **** Greedy Merging Finished ****

	 **** Filling in the rxr-triangular Matrix ****
	 **** Start Merging ****
	 **** Greedy Merging Finished ****

	 **** Filling in the rxr-triangular Matrix ****
	 **** Start Merging ****
	 **** Greedy Merging Finished ****

	 **** Filling in the rxr-triangular Matrix ****
	 **** Start Merging ****
	 **** Greedy Merging Finishe

# Visualization